In [ ]:
from transformers import BertTokenizer, BertForMaskedLM, BertTokenizer, BertForSequenceClassification

def load_model(name):
    model_path = name
    model = BertForSequenceClassification.from_pretrained(
        model_path,
        output_attentions=False
    )
    return model.cuda()
tokenizer_path = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(tokenizer_path, do_lower_case=True)

models = [f"./finetuned-scrumbled-wikitext2/checkpoint-{i}" for i in range(500, 27501, 500)]

In [ ]:
import pandas as pd

df_train = pd.read_csv("target_problem/train.csv")
df_test = pd.read_csv("target_problem/test.csv")

In [ ]:
train_encodings = tokenizer(df_train["text"].values.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(df_test["text"].values.tolist(), truncation=True, padding=True)

In [ ]:
import pandas as pd
import torch
import datasets

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, df_train["label"])
test_dataset = IMDbDataset(test_encodings, df_test["label"])

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    print(eval_pred)
    predictions, labels = eval_pred
    predictions = predictions[0].argmax(axis=1)
    print(predictions)
    res = accuracy.compute(predictions=predictions, references=labels)
    print(res)
    return res

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm



test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)


def evaluate(model, loader):
    model.eval()
    all_pred = []
    all_true = []
    for batch in tqdm(loader):
        input_ids = batch['input_ids'].cuda()
        attention_mask = batch['attention_mask'].cuda()
        labels = batch['labels'].cuda()
        output = model(input_ids, attention_mask=attention_mask, labels=labels)
        predicts = output["logits"].argmax(axis=1)
        all_pred.extend(predicts.cpu().numpy().tolist())
        all_true.extend(batch["labels"].cpu().numpy().tolist())
    return (np.array(all_pred) == np.array(all_true)).astype(float).mean()

open("data.csv", "w").close()
for model_name in models:
    model = load_model(model_name).cuda()

    training_args = TrainingArguments(
        output_dir=f"res/{model_name}",
        learning_rate=2e-5,
        num_train_epochs=3,
        weight_decay=0.01,
        eval_accumulation_steps=2,
        warmup_steps=500,
        per_device_train_batch_size=16,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        # compute_metrics=compute_metrics,
    )
    


    res = trainer.train()
    model = trainer.model
    del trainer
    torch.cuda.empty_cache()

    score = evaluate(model, test_loader)
    with open("data.csv", "a") as fout:
        print(model_name, score, sep="\t", file=fout)

In [ ]:

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, BertForSequenceClassification, BertConfig
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm

test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)


def evaluate(model, loader):
    model.eval()
    all_pred = []
    all_true = []
    for batch in tqdm(loader):
        input_ids = batch['input_ids'].cuda()
        attention_mask = batch['attention_mask'].cuda()
        labels = batch['labels'].cuda()
        output = model(input_ids, attention_mask=attention_mask, labels=labels)
        predicts = output["logits"].argmax(axis=1)
        all_pred.extend(predicts.cpu().numpy().tolist())
        all_true.extend(batch["labels"].cpu().numpy().tolist())
    return (np.array(all_pred) == np.array(all_true)).astype(float).mean()

model = BertForSequenceClassification(BertConfig(num_labels=100))

training_args = TrainingArguments(
    output_dir=f"res/{model_name}",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_accumulation_steps=2,
    warmup_steps=500,
    per_device_train_batch_size=16,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    # compute_metrics=compute_metrics,
)



res = trainer.train()
model = trainer.model
del trainer
torch.cuda.empty_cache()

score = evaluate(model, test_loader)

In [ ]:
score